<br>
<br>
<br>

# Face recognition with `Siamese Network:`

<br>
<br>

<br>

# 1.1 `Import Dependencies:` 

<br>


**Note:** Here, we will use `The Functional Api` In tensorflow.  <br>
`The Keras functional API is a way to create models that are more flexible than the keras.Sequential API.`
<br>

[documentation](https://www.tensorflow.org/guide/keras/functional_api)


In [2]:

# import standard dependencies:

import cv2 as cv 
import os
import random
import numpy as np 
import matplotlib.pyplot as plt 

# import tensorflow for funcitonal api:
import tensorflow as tf

# We are going to do: 
# tf.keras.Model(inputs=(inputImage, varificationImage), outputs=[1,0])


2024-09-03 16:05:56.384362: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-03 16:05:56.764842: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-03 16:05:56.970453: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-03 16:05:57.359296: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-03 16:05:57.482170: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-03 16:05:57.918367: I tensorflow/core/platform/cpu_feature_gu


<br>

# 1.2 `Set Gpu Growth:`

<br>


In [3]:


# i have no gpu on my computer that's why i'm gonna skip this.


<br>

# 1.3 `Create Folder Structure: `

<br>

In [4]:

# define folder name along with their path
# we see this in facenet paper(check README.md), anchor to positive and anchor to negative.

POS_PATH = os.path.join('data','positive')
NEG_PATH = os.path.join('data',"negative")
ANC_PATH = os.path.join('data','anchor')


In [5]:

# create those folder if they are not exist:

if not os.path.exists(POS_PATH):
    os.makedirs(POS_PATH)
if not os.path.exists(NEG_PATH):
    os.makedirs(NEG_PATH)
if not os.path.exists(ANC_PATH):
    os.makedirs(ANC_PATH)





<br>

# 2. `Collect Positive and Anchors: `

<br>

## 2.1 `Untar Labelled Faces in the Wild Dataset:`

[wild_data_set_link](https://vis-www.cs.umass.edu/lfw/)


- We download the LTF(Fabelled Faces in the Wild Dataset in tgz format: ) 
- Now, extract that:
- we are not implementing tripet loss in that case the image folder will be different .  


In [6]:
# extract this file: 
!tar -xf lfw.tgz 

# we get a folder named lfw_funneled

In [7]:

# here, we will find some folder endwith>> '.txt' just delete that files

for file in os.listdir('lfw_funneled'):
    if file.endswith('.txt'):
        os.remove(os.path.join('lfw_funneled',file))
    

In [8]:

# Now, move all the images folder into data/negative folder:

for directory in os.listdir('lfw_funneled'): # going through the all folder lfw_funneled:
    for file in os.listdir(os.path.join('lfw_funneled', directory)): # from that folder going through all the image file:
        EX_PATH = os.path.join('lfw_funneled', directory, file) # create path 
        NEW_PATH = os.path.join(NEG_PATH,file) # create new path 
        os.replace(EX_PATH,NEW_PATH) # now, replace them


In [9]:

# after replace that we will delte the folder lfw_funneled:



<br>

# 2.2 `Collect Positive And Anchor Classes:`


#### Note That: lfw Data set  image dimention is about (250x250): 

<br>

In [21]:
# we gonna use uuid: to generate unique image name:

import uuid

print("uuid: one ",uuid.uuid1())


uuid: one  1a53f926-69e0-11ef-83ba-f48c50632a09


In [22]:
uuid??

Type:        module
String form: <module 'uuid' from '/usr/lib/python3.12/uuid.py'>
File:        /usr/lib/python3.12/uuid.py
Source:     
r"""UUID objects (universally unique identifiers) according to RFC 4122.

This module provides immutable UUID objects (class UUID) and the functions
uuid1(), uuid3(), uuid4(), uuid5() for generating version 1, 3, 4, and 5
UUIDs as specified in RFC 4122.

If all you want is a unique ID, you should probably call uuid1() or uuid4().
Note that uuid1() may compromise privacy since it creates a UUID containing
the computer's network address.  uuid4() creates a random UUID.

Typical usage:

    >>> import uuid

    # make a UUID based on the host ID and current time
    >>> uuid.uuid1()    # doctest: +SKIP
    UUID('a8098c1a-f86e-11da-bd1a-00112444be1e')

    # make a UUID using an MD5 hash of a namespace UUID and a name
    >>> uuid.uuid3(uuid.NAMESPACE_DNS, 'python.org')
    UUID('6fa459ea-ee8a-3ca4-894e-db77e160355e')

    # make a random UUID
    >>> uu

In [36]:
cap = cv.VideoCapture(0)

while(cap.isOpened()):
    ret,frame = cap.read()
    if not ret:
        break
    if cv.waitKey(1) & 0XFF == ord('q'):
        break
    
    frame = cv.flip(frame,1)
    frame = cv.resize(frame,(250,250))
    
    # ---------- for collection anchor --------------
    if cv.waitKey(1) & 0xff == ord('a'):
        unique_image_name = os.path.join(ANC_PATH,"{}.jpg".format(uuid.uuid1()))
        print(type(unique_image_name))
        cv.imwrite(unique_image_name,frame)
        
    # ---------- for collection positive -------------
    if cv.waitKey(1) & 0xff == ord('p'):
        unique_image_name = os.path.join(POS_PATH,"{}.jpg".format(uuid.uuid1()))
        cv.imwrite(unique_image_name,frame)
    cv.imshow("Image Collection",frame)
    
cap.release()
cv.destroyAllWindows()


<br>

# 3. `load and pre-process image:`

<br>

<br>

### 3.1 `Get image directory: `
<br>

<br>

### 3.2 `Preprocessing - Scale and Resize: `
<br>

<br>

### 3.3 `Create label dataset: `
<br>

<br>

### 3.4 `Build train and test partition: `
<br>